In [2]:
import grpc 

import resources_pb2
import moves_pb2
import board_pb2
import game_data_core_pb2 as core


channel = grpc.insecure_channel('localhost:9000')

In [3]:
import game_recorder_service_pb2
import game_recorder_service_pb2_grpc

data_recorder_stub = game_recorder_service_pb2_grpc.GameRecorderServiceStub(channel)

In [5]:
platform = 'test'

playerIds = {
    0 : core.PlayerId(platform = 'test', user_id = 'player1'),
    1 : core.PlayerId(platform = 'test', user_id = 'player2'),
    2 : core.PlayerId(platform = 'test', user_id = 'player3'),
    3 : core.PlayerId(platform = 'test', user_id = 'player4')
}

board = board_pb2.BaseBoard()

gameId = data_recorder_stub.StartRecording(
    game_recorder_service_pb2.StartRecordingRequest(player_position_id_map = playerIds, board = board))

print(gameId)

id: "6"



In [6]:
first_move = moves_pb2.InitialPlacement(vertex = board_pb2.Vertex(id = '1'), edge = board_pb2.Edge(v1 = board_pb2.Vertex(id = '1'), v2 = board_pb2.Vertex(id = '2')))


move_num = data_recorder_stub.RecordMove(
    game_recorder_service_pb2.RecordMoveRequest(id = gameId, move = core.MoveEvent(initial_placement_move = first_move), previous_move_number = 0))

In [7]:
moves = [
    core.MoveEvent(initial_placement_move = moves_pb2.InitialPlacement()),
    core.MoveEvent(initial_placement_move = moves_pb2.InitialPlacement()),
    core.MoveEvent(initial_placement_move = moves_pb2.InitialPlacement()),
    core.MoveEvent(initial_placement_move = moves_pb2.InitialPlacement()),
    core.MoveEvent(initial_placement_move = moves_pb2.InitialPlacement()),
    core.MoveEvent(initial_placement_move = moves_pb2.InitialPlacement()),
    core.MoveEvent(initial_placement_move = moves_pb2.InitialPlacement()),
    
    core.MoveEvent(roll_dice_move = moves_pb2.RollDice()),
    core.MoveEvent(end_turn = moves_pb2.EndTurn())
    # ... all moves
]

for move in moves: 
    move_num = data_recorder_stub.RecordMove(
        game_recorder_service_pb2.RecordMoveRequest(
            id = gameId, 
            move = core.MoveEvent(initial_placement_move = first_move), 
            previous_move_number = move_num.move_number))
print(move_num)

move_number: 10



In [8]:
result = data_recorder_stub.CompleteRecording(game_recorder_service_pb2.CompleteRecordingRequest(id = gameId, num_moves = move_num.move_number))
print(result)

move_count: 10



In [9]:
import game_data_service_pb2
import game_data_service_pb2_grpc

data_access_stub = game_data_service_pb2_grpc.GameDataServiceStub(channel)

In [10]:
data_access_stub.GetGameById(game_data_service_pb2.GetGameInfoByIdRequest(platform = platform, id = gameId))

id {
  id: "6"
}
playerIds {
  key: 0
  value {
    user_id: "player1"
  }
}
playerIds {
  key: 1
  value {
    user_id: "player2"
  }
}
playerIds {
  key: 2
  value {
    user_id: "player3"
  }
}
playerIds {
  key: 3
  value {
    user_id: "player4"
  }
}
board {
}

In [ ]:
data_access_stub.GetRawGameMoves(game_data_service_pb2.GetRawGameMovesRequest(platform = platform, id = gameId))